## Train polyphony encoding

In [1]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
from mukkeBude.model import MukkeBudeTransformer
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import keras

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

We use the music21 corpus for the bach training data. </br>
You can adjust the `paths` to reduce the number of training songs. </br>
</br>
See: https://web.mit.edu/music21/doc/about/referenceCorpus.html

To load custom training data use:
```python
# Load songs
from pathlib import Path

paths = list(Path("./dataset/Pokemon").rglob("*.midi"))
```

In [5]:
from pathlib import Path
paths = list(Path("../mukkeBude/songs/pokemon").rglob("*.mid"))
# Load songs
# paths = m21.corpus.getComposer('bach')
# paths = paths[:100]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for path in paths:
    song = utils.read_single(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.append(mapping.textify(encoded_song))

print(f"Songs encoded: {len(encoded_songs)}")

Found 16 songs in corpus.


c:\Users\flori\anaconda3\envs\tf\lib\site-packages\mukkeBude\utils.py:341: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
c:\Users\flori\anaconda3\envs\tf\lib\site-packages\mukkeBude\utils.py:342: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Chord").highestTime,
c:\Users\flori\anaconda3\envs\tf\lib\site-packages\mukkeBude\utils.py:341: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
c:\Users\flori\anaconda3\envs\tf\lib\site-packages\mukkeBude\utils.py:342: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Chord").highestTime,
In c:\Users\flori\anac

Songs encoded: 16


In [6]:
# Create dataset
utils.create_train_data(encoded_songs, "raw_train_ds.txt")
print("Dataset created")

Dataset created


In [7]:
# Train model
model = MukkeBudeTransformer(mapping)
print(model)

logdir = "logs/bach_transformer"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train("raw_train_ds.txt", min_training_seq_len=32, epochs=50, tensorboard_callback=tensorboard_callback)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        780288    
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 transformer_decoder_2 (Tran  (None, None, 256)        394749

In [8]:
%tensorboard --logdir logs/bach_transformer

In [9]:
model.save("Videospielmusik_polyphonie_transformer")

'c:\\Users\\flori\\anaconda3\\envs\\tf\\lib\\site-packages\\mukkeBude\\model\\preTrainedModels\\Videospielmusik_polyphonie_transformer.h5'

## Generate polyphony

In [22]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [23]:
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import tensorflow as tf
import numpy as np
from pathlib import Path

from mukkeBude.mapping import SPECIAL_TOKS
from mukkeBude.mapping import SEP
from mukkeBude.mapping import BOS

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [24]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

**This does not work! In the current state you can not load a transformer network from a file!**</br>
You need to train it first and use it here.

In [25]:
#model = MukkeBudeTransformer.load(mapping, "Bach_polyphonie_transformer")

In [26]:
# Create song
generated_song = model.generate("n46 d4 xxsep d4 n53 d2 n50 d2 xxsep d2 n62 d1 n58 d1 xxsep d2 n53 d2 n53 d2 n50 d2 xxsep d4 n46 d4 xxsep d4 n55 d2 n51 d2 xxsep d2 n63 d1 n58 d1 xxsep d2 n55 d2 n55 d2 n51 d2 xxsep d4 n46 d4 xxsep d4 n57 d2 n53 d2 xxsep d2 n65 d1 n60 d1 xxsep d2 n57 d2 n57 d2 n53 d2 xxsep d4 n46 d4 xxsep d4 n55 d2 n51 d2 xxsep d2 n63 d1 n58 d1 xxsep d2 n55 d2 n55 d2 n51 d2 xxsep d4 n77 d2 n74 d2 n46 d4 xxsep d4 n75 d1 n72 d1 n53 d2 n50 d2 xxsep d2 n74 d2 n70 d2 xxsep d2 n53 d2 n50 d2 xxsep d2 n75 d1 n72 d1 xxsep d2 n46 d4 xxsep d2 n74 d2 n70 d2 xxsep d2 n53 d2 n50 d2 xxsep d2 n70 d1 n65 d1 xxsep d2 n65 d2 n62 d2 n53 d2 n50 d2 xxsep d4 n67 d2 n63 d2 n46 d4 xxsep d4 n65 d1 n62 d1 n55 d2 n51 d2 xxsep d2 n67 d2 n63 d2 xxsep d2 n55 d2 n51 d2 xxsep d2 n69 d1 n60 d1 xxsep d2 n70 d1 n62 d1 n46 d4 xxsep d2 n77 d1 n70 d1 xxsep d2 n53 d2 n50 d2 xxsep d4 n53 d2 n50 d2 xxsep d3 n77 d1 n75 d1 xxsep d1 n79 d2 n75 d2 n46 d4 xxsep d4 n77 d1 n74 d1 n55 d2 n51 d2 xxsep d2 n79 d2 n75 d2 xxsep d2 n55 d2 n51 d2 xxsep d2 n81 d1 n72 d1 xxsep d2 n82 d2 n74 d2 n46 d4 xxsep d4 n77 d1 n70 d2 n53 d2 n50 d2 xxsep d2 n65 d2 xxsep d2 n53 d2 n50 d2 xxsep d2 n62 d1 xxsep d2 n75 d2 n39 d4 xxsep d4 n74 d1 n62 d1 n51 d2 n46 d2 xxsep d2 n72 d2 n63 d2 xxsep d2 n51 d2 n46 d2 xxsep d2 n70 d1 n64 d1 n52 d2 xxsep d2 n79 d2 n65 d2 n41 d4 xxsep d4 n69 d1 n53 d2 n48 d2 xxsep d2 n77 d1 n72 d2 xxsep d2 n75 d1 n53 d2 n48 d2 xxsep d2 n74 d1 n65 d1 xxsep d1 n72 d1 xxsep d1 n70 d2 n67 d2 n63 d2 n51 d4 n39 d4 xxsep d4 n58 d1 n55 d2 n51 d2 n46 d2 xxsep d2 n70 d2 n67 d2 n63 d2 xxsep d2 n55 d2 n51 d2 n46 d2 xxsep d2 n58 d1 xxsep d2 n70 d2 n65 d2 n62 d2 n46 d4 n34 d4 xxsep d4 n58 d1 n58 d2 n53 d2 n50 d2 xxsep d2 n70 d2 n65 d2 n62 d2 xxsep d2 n58 d2 n53 d2 n50 d2 xxsep d2 n58 d1 xxsep d2 n77 d2 n69 d2 n53 d4 n41 d4 xxsep d4 n75 d1 n57 d2 n53 d2 n48 d2 xxsep d2 n74 d2 n65 d2 xxsep d2 n57 d2 n53 d2 n48 d2 xxsep d2 n72 d1 xxsep d2", max_length=1000, probability=0.4)

# Remove REST and WAIT_LSTM from SPECIAL_TOKS
special_tokens = SPECIAL_TOKS.copy()
special_tokens.remove(SEP)
special_tokens.remove(BOS)

generated_song = " ".join(utils.replace_special_tokens(generated_song.split(), "d1", special_tokens))
print(generated_song)

ValueError: 

In [ ]:
# Convert to music21
new_song_ints = mapping.numericalize(generated_song.split(" "))
new_song_ints = np.array(new_song_ints)

new_song = utils.from_polyphonic_encoding(new_song_ints, mapping, bpm=140)

path = Path("generated_song_pokemon_trans_poly.midi")
utils.write_midi(new_song, path)
